In [6]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, accuracy_score


In [7]:
df_train = pd.read_csv("input/mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("input/mitbih_test.csv", header=None)

Y = np.array(df_train[187].values)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]


In [8]:
BATCH_SIZE = 128
NUM_RESIDUAL_BLOCKS = 5


In [9]:
class CustomDataset(Dataset):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        return torch.tensor(self.x[index], dtype=torch.double), torch.tensor(self.y[index])

train_dataset = CustomDataset(X, Y)
val_dataset = CustomDataset(X_test, Y_test)

def load_data(train_dataset, val_dataset):

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
    
    return train_loader, val_loader

train_loader, val_loader = load_data(train_dataset, val_dataset)


In [65]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device_count = torch.cuda.device_count()
# device = 'cpu'

In [104]:
class CNResidual(nn.Module):
    def __init__(self, size):
        super().__init__()
        # layers
        self.c1 = nn.Conv1d(size, size, kernel_size=5, padding=2)
        self.c2 = nn.Conv1d(size, size, kernel_size=5, padding=2)
        self.pool = nn.MaxPool1d(kernel_size=5, stride=2)
        
    def forward(self, x):
        # call layers
        x1 = F.relu(self.c1(x))
        x1 = self.c2(x1)
        return self.pool(F.relu(x+x1))
    
class CNet(nn.Module):

    def __init__(self):
        super().__init__()
        # layers
        self.c1 = nn.Conv1d(1, 32, kernel_size=5, padding=2)
        self.cresid1 = CNResidual(32)
        self.cresid2 = CNResidual(32)
        self.cresid3 = CNResidual(32)
        self.cresid4 = CNResidual(32)
        self.cresid5 = CNResidual(32)
#         self.cresids = []
#         for i in range(NUM_RESIDUAL_BLOCKS):
#             self.cresids.append(CNResidual(32).to(device))
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 5)
        
    def forward(self, x):
        # call layers
        x = self.c1(x)
        x = self.cresid1(x)
        x = self.cresid2(x)
        x = self.cresid3(x)
        x = self.cresid4(x)
        x = self.cresid5(x)
#         for cresid in self.cresids:
#             x = cresid(x)
        x  = x.reshape(-1, 64)
        x = F.relu(self.fc1(x))
        return F.softmax(self.fc2(x), dim=1)

cnetmodel = CNet()
# if device_count > 1:
#     cnetmodel = nn.DataParallel(cnetmodel)
cnetmodel = cnetmodel.to(device)

In [105]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnetmodel.parameters(), lr=1e-1)


In [106]:
def eval(model, val_loader):
    model.eval()

    x_pred = []
    x_true = []
    i = 0
    for x, y in val_loader:
        x=x.to(device).reshape(-1, 1, 187).float()
        x_hat = model(x)
        x_hat = torch.argmax(x_hat, dim=1)
        x_pred.extend(x_hat.cpu().numpy())
        x_true.extend(y.long().detach().numpy())
    f = f1_score(y_pred=x_pred, y_true=x_true, average='micro')
    acc = accuracy_score(y_pred=x_pred, y_true=x_true)
    return f, acc


In [107]:
def train(model, train_loader, val_loader, n_epochs):
    model.train()
    for epoch in range(n_epochs):
        train_loss = 0
        i=0
        for x, y in train_loader:
            x=x.to(device).reshape(-1, 1, 187).float()
            y = y.to(device).long()
            x_hat = model(x)
            loss = criterion(x_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))
        f, acc = eval(model, val_loader)
        print('Epoch: %d \t Validation f: %.2f, acc: %.2f'%(epoch+1, f, acc))

In [108]:
train(cnetmodel, train_loader, val_loader,2 )


Epoch: 1 	 Training Loss: 1.077601
Epoch: 1 	 Validation f: 0.83, acc: 0.83
Epoch: 2 	 Training Loss: 1.077574
Epoch: 2 	 Validation f: 0.83, acc: 0.83


In [60]:
x, y =next(iter(train_loader))
x=x.reshape(-1, 1, 187).float()

In [61]:
y.long()

tensor([0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 4, 0, 0, 0, 2, 0, 2,
        0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 1, 2, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 4, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0])

In [63]:
x_hat = cnetmodel(x)
torch.argmax(x_hat, dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])

In [72]:
cnetmodel

CNet(
  (c1): Conv1d(1, 32, kernel_size=(5,), stride=(1,), padding=(2,))
  (cresid1): CNResidual(
    (c1): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
    (c2): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
    (pool): MaxPool1d(kernel_size=5, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (cresid2): CNResidual(
    (c1): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
    (c2): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
    (pool): MaxPool1d(kernel_size=5, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (cresid3): CNResidual(
    (c1): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
    (c2): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
    (pool): MaxPool1d(kernel_size=5, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (cresid4): CNResidual(
    (c1): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
    (c2): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=